In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pathlib, h5py, os

In [ ]:
epochs = 20
batch_size = 512
learning_rate = 0.00001
accs = [0.0, 0.01]
rate_changes= []
saveaddress = pathlib.Path("F:\ML\Tabriz Uni\\models\\trained")


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

    # Currently, memory growth needs to be the same across GPUs
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

In [ ]:
train = read_dataset = pd.read_excel("F:\ML\Tabriz Uni\Phase Full Norm.xlsx", sheet_name="Phase", engine='openpyxl')
train = train.sample(frac=1)
try:
    train.drop(columns="Unnamed: 0",inplace=True)
except:
    print("no unnamed")
train_features = np.expand_dims(train.iloc[:13000, 0:14].to_numpy(), 2)
train_labels = np.expand_dims(train.iloc[:13000, 14:28].to_numpy(), 2)
test_features = np.expand_dims(train.iloc[13000:, 0:14].to_numpy(), 2)
test_labels = np.expand_dims(train.iloc[13000:, 14:28].to_numpy(), 2)
print("train:", train_features.shape, train_labels.shape, " test:", test_features.shape, test_labels.shape)

In [60]:
train = read_dataset = pd.read_excel("F:\ML\Tabriz Uni\Phase norm train mixed.xlsx", sheet_name="Phase", engine='openpyxl')
train_features = np.expand_dims(train.iloc[:, 0:14].to_numpy(), 2)
train_labels = np.expand_dims(train.iloc[:, 14:28].to_numpy(), 2)

test = pd.read_excel("F:\ML\Tabriz Uni\Phase norm test mixed.xlsx", sheet_name="Phase", engine='openpyxl')
test_features = np.expand_dims(test.iloc[:, 0:14].to_numpy(), 2)
test_labels = np.expand_dims(test.iloc[:, 14:28].to_numpy(), 2)

print("train:", train_features.shape, train_labels.shape, " test:", test_features.shape, test_labels.shape)
#print(test_features[0])

train: (13709, 14, 1) (13709, 14, 1)  test: (2419, 14, 1) (2419, 14, 1)


In [17]:
#0.0002 noise to original data brings 0,00001ms or 1e-8 s delay in 50Hz

In [61]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_features)
train_label = tf.data.Dataset.from_tensor_slices(train_labels)

test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))
test_dataset = test_dataset.batch(batch_size)

In [5]:
model_specs = {"first" : [8, 12, 16], "second": [2, 4], "smaller" : [True, False],
               "third" : [1, 2], "dropout" : [0, 0.2, 0.4, 0.6]}
combinations = []
#for a in model_specs["first"]:
for b in model_specs["second"]:
    #if b<a:
    for c in model_specs["third"]:
        if c<b:
            #for d in model_specs["smaller"]:
            combinations.append([b,c])
print(len(combinations))

3


In [34]:
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, UpSampling1D

def modelMaker():
    input_layer = tf.keras.Input(shape= (14,1,))
    conv1 = Conv1D(filters=16, kernel_size=3, padding='same', activation='relu')(input_layer)
    maxp1 = MaxPooling1D(2)(conv1)
    conv2 = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu')(maxp1)
    
    maxp2 = MaxPooling1D(2, name='encoder')(conv2)
    
    up1 = UpSampling1D(2)(maxp2)
    conv3 = Conv1D(filters=8, kernel_size=3, padding='valid', activation='relu')(up1)
    up2 = UpSampling1D(2)(conv3)
    conv4 = Conv1D(filters=16, kernel_size=3, padding='same', activation='relu')(up2)
    up3 = UpSampling1D(2)(conv4)
    out = Conv1D(filters=1, kernel_size=3, padding='valid', activation='relu', name='decoder')(up3)
    

 
    model = tf.keras.Model(input_layer, out)
    model.summary(line_length = 60)
    return model

In [35]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor='accuracy', value=0.99, verbose=1):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        accs.append(current)
        if current > self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping " % epoch, " acc=", current)
            self.model.stop_training = True
            self.model.save(str(save_folder) + "\\best model on epoch {0}".format(epoch))
            accs.append(0.00)
            accs.append(0.01)
        
        #train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
        #train_dataset = train_dataset.shuffle(1000).batch(batch_size)
            

In [71]:

save_folder = saveaddress / "33000"
print(save_folder)
try:
    os.mkdir(save_folder)
except:
    print("folder already exists")

csv_log = tf.keras.callbacks.CSVLogger(str(save_folder/ "epochresults.csv"))
#model = modelMaker()

#opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer= opt, loss="mse", metrics=['accuracy'])
model.fit(train_features, train_labels, validation_data=test_dataset, batch_size=batch_size,
          epochs=3000, shuffle=True, callbacks=[CustomCallback(), csv_log])

model.save(str(save_folder / "final model.h5"))
accs.append(0.00)
accs.append(0.01)

F:\ML\Tabriz Uni\models\trained\33000
Epoch 1/3000
27/27 [==============================] - 0s 11ms/step - loss: 4.0240e-05 - accuracy: 0.0000e+00 - val_loss: 3.9222e-05 - val_accuracy: 0.0000e+00
Epoch 2/3000
27/27 [==============================] - 0s 5ms/step - loss: 4.0207e-05 - accuracy: 0.0000e+00 - val_loss: 3.9237e-05 - val_accuracy: 0.0000e+00
Epoch 3/3000
27/27 [==============================] - 0s 5ms/step - loss: 4.0206e-05 - accuracy: 0.0000e+00 - val_loss: 3.9187e-05 - val_accuracy: 0.0000e+00
Epoch 4/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0177e-05 - accuracy: 0.0000e+00 - val_loss: 3.9182e-05 - val_accuracy: 0.0000e+00
Epoch 5/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0198e-05 - accuracy: 0.0000e+00 - val_loss: 3.9304e-05 - val_accuracy: 0.0000e+00
Epoch 6/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0232e-05 - accuracy: 0.0000e+00 - val_loss: 3.9189e-05 - val_accuracy: 0.0000e+00
Epoch 7/3000


27/27 [==============================] - 0s 4ms/step - loss: 4.0185e-05 - accuracy: 0.0000e+00 - val_loss: 3.9225e-05 - val_accuracy: 0.0000e+00
Epoch 104/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0185e-05 - accuracy: 0.0000e+00 - val_loss: 3.9199e-05 - val_accuracy: 0.0000e+00
Epoch 105/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0198e-05 - accuracy: 0.0000e+00 - val_loss: 3.9232e-05 - val_accuracy: 0.0000e+00
Epoch 106/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0190e-05 - accuracy: 0.0000e+00 - val_loss: 3.9190e-05 - val_accuracy: 0.0000e+00
Epoch 107/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0190e-05 - accuracy: 0.0000e+00 - val_loss: 3.9200e-05 - val_accuracy: 0.0000e+00
Epoch 108/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0195e-05 - accuracy: 0.0000e+00 - val_loss: 3.9192e-05 - val_accuracy: 0.0000e+00
Epoch 109/3000
27/27 [==============================] -

27/27 [==============================] - 0s 4ms/step - loss: 4.0175e-05 - accuracy: 0.0000e+00 - val_loss: 3.9174e-05 - val_accuracy: 0.0000e+00
Epoch 206/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0184e-05 - accuracy: 0.0000e+00 - val_loss: 3.9181e-05 - val_accuracy: 0.0000e+00
Epoch 207/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0197e-05 - accuracy: 0.0000e+00 - val_loss: 3.9200e-05 - val_accuracy: 0.0000e+00
Epoch 208/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0187e-05 - accuracy: 0.0000e+00 - val_loss: 3.9181e-05 - val_accuracy: 0.0000e+00
Epoch 209/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0182e-05 - accuracy: 0.0000e+00 - val_loss: 3.9176e-05 - val_accuracy: 0.0000e+00
Epoch 210/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0178e-05 - accuracy: 0.0000e+00 - val_loss: 3.9175e-05 - val_accuracy: 0.0000e+00
Epoch 211/3000
27/27 [==============================] -

27/27 [==============================] - 0s 4ms/step - loss: 4.0168e-05 - accuracy: 0.0000e+00 - val_loss: 3.9191e-05 - val_accuracy: 0.0000e+00
Epoch 308/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0181e-05 - accuracy: 0.0000e+00 - val_loss: 3.9162e-05 - val_accuracy: 0.0000e+00
Epoch 309/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0169e-05 - accuracy: 0.0000e+00 - val_loss: 3.9187e-05 - val_accuracy: 0.0000e+00
Epoch 310/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0173e-05 - accuracy: 0.0000e+00 - val_loss: 3.9177e-05 - val_accuracy: 0.0000e+00
Epoch 311/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0181e-05 - accuracy: 0.0000e+00 - val_loss: 3.9197e-05 - val_accuracy: 0.0000e+00
Epoch 312/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0179e-05 - accuracy: 0.0000e+00 - val_loss: 3.9231e-05 - val_accuracy: 0.0000e+00
Epoch 313/3000
27/27 [==============================] -

27/27 [==============================] - 0s 4ms/step - loss: 4.0169e-05 - accuracy: 0.0000e+00 - val_loss: 3.9176e-05 - val_accuracy: 0.0000e+00
Epoch 410/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0185e-05 - accuracy: 0.0000e+00 - val_loss: 3.9202e-05 - val_accuracy: 0.0000e+00
Epoch 411/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0189e-05 - accuracy: 0.0000e+00 - val_loss: 3.9235e-05 - val_accuracy: 0.0000e+00
Epoch 412/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0209e-05 - accuracy: 0.0000e+00 - val_loss: 3.9196e-05 - val_accuracy: 0.0000e+00
Epoch 413/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0174e-05 - accuracy: 0.0000e+00 - val_loss: 3.9187e-05 - val_accuracy: 0.0000e+00
Epoch 414/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0170e-05 - accuracy: 0.0000e+00 - val_loss: 3.9167e-05 - val_accuracy: 0.0000e+00
Epoch 415/3000
27/27 [==============================] -

...

Epoch 2755/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0113e-05 - accuracy: 0.0000e+00 - val_loss: 3.9094e-05 - val_accuracy: 0.0000e+00
Epoch 2756/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0102e-05 - accuracy: 0.0000e+00 - val_loss: 3.9103e-05 - val_accuracy: 0.0000e+00
Epoch 2757/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0100e-05 - accuracy: 0.0000e+00 - val_loss: 3.9134e-05 - val_accuracy: 0.0000e+00
Epoch 2758/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0093e-05 - accuracy: 0.0000e+00 - val_loss: 3.9111e-05 - val_accuracy: 0.0000e+00
Epoch 2759/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0123e-05 - accuracy: 0.0000e+00 - val_loss: 3.9133e-05 - val_accuracy: 0.0000e+00
Epoch 2760/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0103e-05 - accuracy: 0.0000e+00 - val_loss: 3.9092e-05 - val_accuracy: 0.0000e+00
Epoch 2761/3000
27/27 [===========

Epoch 2806/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0105e-05 - accuracy: 0.0000e+00 - val_loss: 3.9087e-05 - val_accuracy: 0.0000e+00
Epoch 2807/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0095e-05 - accuracy: 0.0000e+00 - val_loss: 3.9094e-05 - val_accuracy: 0.0000e+00
Epoch 2808/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0100e-05 - accuracy: 0.0000e+00 - val_loss: 3.9118e-05 - val_accuracy: 0.0000e+00
Epoch 2809/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0080e-05 - accuracy: 0.0000e+00 - val_loss: 3.9082e-05 - val_accuracy: 0.0000e+00
Epoch 2810/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0094e-05 - accuracy: 0.0000e+00 - val_loss: 3.9094e-05 - val_accuracy: 0.0000e+00
Epoch 2811/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0123e-05 - accuracy: 0.0000e+00 - val_loss: 3.9161e-05 - val_accuracy: 0.0000e+00
Epoch 2812/3000
27/27 [===========

Epoch 2857/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0082e-05 - accuracy: 0.0000e+00 - val_loss: 3.9119e-05 - val_accuracy: 0.0000e+00
Epoch 2858/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0102e-05 - accuracy: 0.0000e+00 - val_loss: 3.9092e-05 - val_accuracy: 0.0000e+00
Epoch 2859/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0090e-05 - accuracy: 0.0000e+00 - val_loss: 3.9125e-05 - val_accuracy: 0.0000e+00
Epoch 2860/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0092e-05 - accuracy: 0.0000e+00 - val_loss: 3.9127e-05 - val_accuracy: 0.0000e+00
Epoch 2861/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0088e-05 - accuracy: 0.0000e+00 - val_loss: 3.9088e-05 - val_accuracy: 0.0000e+00
Epoch 2862/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0082e-05 - accuracy: 0.0000e+00 - val_loss: 3.9090e-05 - val_accuracy: 0.0000e+00
Epoch 2863/3000
27/27 [===========

Epoch 2908/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0083e-05 - accuracy: 0.0000e+00 - val_loss: 3.9103e-05 - val_accuracy: 0.0000e+00
Epoch 2909/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0089e-05 - accuracy: 0.0000e+00 - val_loss: 3.9079e-05 - val_accuracy: 0.0000e+00
Epoch 2910/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0092e-05 - accuracy: 0.0000e+00 - val_loss: 3.9096e-05 - val_accuracy: 0.0000e+00
Epoch 2911/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0078e-05 - accuracy: 0.0000e+00 - val_loss: 3.9093e-05 - val_accuracy: 0.0000e+00
Epoch 2912/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0083e-05 - accuracy: 0.0000e+00 - val_loss: 3.9122e-05 - val_accuracy: 0.0000e+00
Epoch 2913/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0092e-05 - accuracy: 0.0000e+00 - val_loss: 3.9084e-05 - val_accuracy: 0.0000e+00
Epoch 2914/3000
27/27 [===========

Epoch 2959/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0107e-05 - accuracy: 0.0000e+00 - val_loss: 3.9084e-05 - val_accuracy: 0.0000e+00
Epoch 2960/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0096e-05 - accuracy: 0.0000e+00 - val_loss: 3.9141e-05 - val_accuracy: 0.0000e+00
Epoch 2961/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0082e-05 - accuracy: 0.0000e+00 - val_loss: 3.9085e-05 - val_accuracy: 0.0000e+00
Epoch 2962/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0092e-05 - accuracy: 0.0000e+00 - val_loss: 3.9088e-05 - val_accuracy: 0.0000e+00
Epoch 2963/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0089e-05 - accuracy: 0.0000e+00 - val_loss: 3.9074e-05 - val_accuracy: 0.0000e+00
Epoch 2964/3000
27/27 [==============================] - 0s 4ms/step - loss: 4.0087e-05 - accuracy: 0.0000e+00 - val_loss: 3.9101e-05 - val_accuracy: 0.0000e+00
Epoch 2965/3000
27/27 [===========

In [72]:
outcome =model.predict(test_features[:])
#print(outcome,"and\n", train_labels[0])
print(outcome.shape)

(2419, 14, 1)


In [49]:
model = tf.keras.models.load_model("F:\ML\Tabriz Uni\\models\\trained\\3000\\final model.h5")

In [11]:
#outcome =outcome.reshape((14,1))
print(outcome - test_labels[100:105])

[[[ 1.35355793e-03]
  [ 4.10978564e-03]
  [-4.06871754e-03]
  [ 1.81720901e-03]
  [ 5.19015875e-05]
  [ 2.93535985e-03]
  [-5.62175064e-03]
  [-5.84859987e-03]
  [-4.17039202e-03]
  [-1.11112704e-03]
  [ 5.18308478e-04]
  [-1.56403876e-02]
  [ 8.22498490e-03]
  [ 1.34798744e-02]]

 [[-1.34981748e-03]
  [-3.66735890e-03]
  [-2.24722064e-03]
  [ 2.30696870e-03]
  [-7.57278255e-03]
  [-6.94641359e-03]
  [-8.36238918e-04]
  [ 6.17774706e-04]
  [ 9.56981117e-04]
  [ 2.87651341e-03]
  [-2.62518989e-03]
  [ 9.78912999e-03]
  [-7.34635580e-03]
  [-1.01082160e-02]]

 [[-3.27541585e-03]
  [-1.38699654e-03]
  [ 1.71836180e-03]
  [-3.27534043e-03]
  [-6.63005897e-03]
  [-1.92842339e-03]
  [-2.67316972e-03]
  [-2.20743071e-03]
  [ 5.82241558e-03]
  [ 1.65157061e-03]
  [ 1.23299004e-03]
  [ 7.84358382e-03]
  [-7.89964489e-03]
  [-7.02371661e-03]]

 [[-8.63289482e-04]
  [ 1.57731769e-03]
  [-6.05271091e-03]
  [ 1.43373528e-02]
  [ 1.60718383e-02]
  [-2.76702842e-03]
  [-7.31775149e-04]
  [-1.90542357

In [54]:
def unnormer(df):
    src = pd.read_excel("F:\ML\Tabriz Uni\\minmax all.xlsx", engine="openpyxl")
    #orig_cl = df.columns.values
    #src.columns = orig_cl
    minv = src.iloc[0].to_numpy()
    maxv = src.iloc[1].to_numpy()   
    d_range = (maxv - minv)
    df_unnorm = df * d_range
    df_unnorm = df_unnorm + minv
    return df_unnorm

In [82]:
outcome =outcome.reshape((2419,14))
retest_labels = test_labels.reshape((2419,14))
unnormed_outcome = unnormer(outcome)
unnormed_label = unnormer(retest_labels)
diff = unnormed_outcome - unnormed_label
print(unnormed_outcome[10]- unnormed_label[10])

[-5.10811564e-05  4.79379600e-05  8.13888261e-04 -2.28976782e-04
 -5.28871153e-04  5.19563426e-06 -7.79205448e-04 -8.07464888e-04
 -4.82581112e-04 -3.50080102e-04 -1.89274616e-04  5.73598062e-04
 -4.43077664e-04 -1.14089481e-04]


In [84]:
pred_clms = train.iloc[:,0:14].columns.values
diff_clms = train.iloc[:,14:28].columns.values
outcome_df = pd.DataFrame(unnormed_outcome, columns=pred_clms)
diff_df = pd.DataFrame(diff, columns=diff_clms)

In [85]:
together = outcome_df.join(diff_df)
together.to_excel("F:\ML\Tabriz Uni\pred and diff.xlsx", sheet_name="Phase", engine='openpyxl')